RM1 and RM3, NTLM and Kuzi et al. embedding methods. Includes tuning of NTLM and Kuzi et al. methods and comparison of different embedding types.

In [ ]:
import sys 
import os 
import numpy as np

nb_dir = os.getcwd()
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

from plotlib.loaders import *
from plotlib.plotters import *

from phdconf import config 
from phdconf.config import *

%matplotlib inline

In [ ]:
queries = load_queries(config.AUS_TOPIC_PATH)

In [ ]:
tt_folds = read_folds(AUS_FOLDS)

In [ ]:
index_names = ['case-topics']
qrel_paths = [config.AUS_QREL_PATH] * len(index_names)
rel_levels = [config.AUS_REL_LEVEL] * len(index_names)
# display_names = ['base', 'sec-count', 'leg-count', 'date', 'hierarchy']

topics = 'case-topics'

In [ ]:
mu = 1050
base_df = load_1d_dfs(['filtered-phrasestop'], qrel_paths, os.path.join(BASE_DIR, 'preprocessing', 'dirichlet_prior'), 'case-topics-{0}-unigram_dir_mu_{1:.2f}.run', rel_levels, mu, mu, 1)[0][0]
base_qry = load_1d_dfs(['filtered-phrasestop'], qrel_paths, os.path.join(BASE_DIR, 'preprocessing', 'dirichlet_prior'), 'case-topics-{0}-unigram_dir_mu_{1:.2f}.run', rel_levels, mu, mu, 1, per_query=True)[0][0]

# RM1

In [ ]:
# ['tmp.run']
results = []
for n in range(1, 6):
    results.append(load_dfs(config.AUS_QREL_PATH, config.AUS_REL_LEVEL, '', [os.path.join(BASE_DIR, 'prf', 'case-topics-prf-text-docs-'+str(n)+'-terms-{0}.run'.format(x)) for x in range(1, 11)]))

In [ ]:
doc_fig = plot_tune_1d_comp(['DP']+[x for x in range(1, 6)], RERANK_METRICS, [[base_df for x in range(10)]]+results, 1, 10, 1, legend_x=0.965, legend_y=0.75, styles=['--'])

In [ ]:
# doc_fig.savefig('figures/rm1-docs.pdf')

# RM3

In [ ]:
inter = Interpolater(os.path.join(BASE_DIR, 'preprocessing', 'dirichlet_prior', 'case-topics-filtered-phrasestop-unigram_dir_mu_1050.00.run'), normalize=True)

dfs = []
for x in [os.path.join(BASE_DIR, 'prf', 'case-topics-prf-text-docs-1-terms-{0}.run'.format(x)) for x in range(1, 11)]: 
    interped_dfs = []
    for _lambda in np.arange(0, 1.0, 0.01):
        inter.interpolate(x, _lambda, 'tmp.run')
        interped_dfs.append(load_dfs(config.AUS_QREL_PATH, config.AUS_REL_LEVEL, '', ['tmp.run'])[0])
    dfs.append(interped_dfs)

In [ ]:
to = 30
term_interp_fig = plot_tune_1d_comp(['DP']+[x for x in range(1, 11)], RERANK_METRICS, [[base_df for x in range(to+1)]] + [x[:to+1] for x in dfs], 0.00, (to)/100, 0.01, legend_x=0.73, legend_y=0.93, styles=['--'])

In [ ]:
# term_interp_fig.savefig('figures/rm3.pdf')

In [ ]:
names = [x for x in range(1, 11)]
max_1_doc = select_1d_max_with_interp(names, dfs, 0.0, 0.01, '$\lambda$', inter, base_qry, base_df, 1050, os.path.join(BASE_DIR, 'prf', 'case-topics-prf-text-docs-1-terms-{0}.run'), config.AUS_QREL_PATH, config.AUS_REL_LEVEL, metrics=RERANK_METRICS).T

In [ ]:
print(max_1_doc.drop(['Unjudged@20'], axis='columns').to_latex(escape=False))

In [ ]:
# ['tmp.run']
fixed_lambda = 0.1
results = []
for n in range(1, 6):
    res = []
    for k in range(1, 11):
        inter.interpolate(os.path.join(BASE_DIR, 'prf', 'case-topics-prf-text-docs-{0}-terms-{1}.run'.format(n, k)), fixed_lambda, 'tmp.run')
        res.append(load_dfs(config.AUS_QREL_PATH, config.AUS_REL_LEVEL, '', ['tmp.run'])[0])
    results.append(res)              

In [ ]:
interp_fig = plot_tune_1d_comp(['base']+[x for x in range(1, 6)], RERANK_METRICS, [[base_df for x in range(10)]]+results, 1, 10, 1, legend_x=0.965, ylims=RERANK_YLIMS, styles=['--'])

In [ ]:
# interp_fig.savefig('figures/ausnl-prf-fixed-interp.pdf')

# Kuzi et al methods

In [ ]:
emb_type = 'max'
emb_results = []
for k in range(1, 11):
    res = []
    for _lambda in np.arange(0, 1.0, 0.01):
        inter.interpolate(os.path.join(BASE_DIR, 'embedding-prf', 'case-topics-skip-emb-{0}-expterms-{1}.run'.format(emb_type, k)), _lambda, 'tmp.run')
        res.append(load_dfs(config.AUS_QREL_PATH, config.AUS_REL_LEVEL, '', ['tmp.run'])[0])
    emb_results.append(res)              

In [ ]:
to = 40
interp_fig = plot_tune_1d_comp(['base']+[x for x in range(1, 11)], RERANK_METRICS, [[base_df for x in range(to+1)]]+[x[:to+1] for x in emb_results], 0, to/100, 0.01, legend_x=0.965, ylims=RERANK_YLIMS, styles=['--'])
# interp_fig.savefig('figures/ausnl-emb-interp.pdf')

In [ ]:
grid_dir = os.path.join(BASE_DIR, "grid-search")

models = ['sum', 'mnz', 'max', 'centroid']
ets = ['cbow', 'skip']

names = ['rm3']#, '5feat']#, '6feat']
for e in ets: 
    for m in models: 
        names.append('{0}-{1}'.format(e, m))
        names.append('rm3-{0}-{1}'.format(e, m))

grid_name = os.path.join(grid_dir, '{0}-{1}-combine-max.run')
scores = {}
run_res = {}
for name in names:
    run_res[name] = {}
    scores[name] = {}
    for m, metric in zip(['RR', 'ERR', 'R@20', 'NDCG', 'RBP'], RERANK_METRICS):
        run = load_dfs(config.AUS_QREL_PATH, config.AUS_REL_LEVEL, '', [grid_name.format(name, m)], per_query=True)[0]
        run_res[name][metric] = run[metric]
        scores[name][metric] = run[metric].mean()

In [ ]:
from scipy import stats 

base_qry.sort_index(inplace=True)

stat = {}
for name, rr in scores.items():
    stat[name] = {}
    for metric, score in rr.items():
        run_res[name][metric].sort_index(inplace=True)
        p = stats.ttest_rel(run_res[name][metric], base_qry[metric]).pvalue
        m = base_qry[metric].mean()
        if p < 0.01:
             stat[name][metric] = '{0:.4f}'.format((score)) + "$^{**}$"
        elif p < 0.05: 
            stat[name][metric] = '{0:.4f}'.format((score)) + "$^{*}$"
        else:
            stat[name][metric] = '{0:.4f}'.format((score))

In [ ]:
df = pd.DataFrame(stat).T

In [ ]:
df.loc['$R$'] = base_df[RERANK_METRICS].round(4)
df = df.rename(index={'rm3': 'RM3'})
df = df.reindex(['$R$', 'RM3', 'cbow-centroid', 'cbow-sum', 'cbow-max', 'cbow-mnz', 
                 'rm3-cbow-centroid', 'rm3-cbow-sum', 'rm3-cbow-max', 'rm3-cbow-mnz',
                'skip-centroid', 'skip-sum', 'skip-max', 'skip-mnz',
                'rm3-skip-centroid', 'rm3-skip-sum', 'rm3-skip-max', 'rm3-skip-mnz'])
df
# write_table('tables/emb-expansion', bold_max(df).rename(columns=metrics).to_latex(escape=False))

# NTLM

In [ ]:
ntlm_emb = ['all-paras-lower-100-skip-minoccur-50', 'all-paras-lower-100-cbow-minoccur-50', 'filtered-wiki-news-300d-1M']
abbrev = ['skip', 'cbow', 'generic']
ntlm_runs = []
for emb, ab in zip(ntlm_emb, abbrev):
    ntlm_names = [os.path.join(BASE_DIR, 'expansion/tlm', 'case-topics-filtered-phrasestop-{0}-tlm-dir-mu-1050.00-neighbours-{1}.run'.format(emb, x)) for x in range(1, 21)]
    ntlm_runs.append(load_dfs(config.AUS_QREL_PATH, config.AUS_REL_LEVEL, '', ntlm_names, per_query=True))


In [ ]:
ntlm_df = pd.DataFrame(columns=RERANK_METRICS)

for ab, runs in zip(abbrev, ntlm_runs):
    cross = cross_validation(runs, tt_folds, RERANK_METRICS, base_qry)
    ntlm_df.loc[ab] = cross[0]
    
ntlm_df.loc['$R$'] = base_df.round(4)
ntlm_df = ntlm_df.reindex(['$R$', 'skip', 'cbow', 'generic'])

In [ ]:
# write_table('tables/ntlm-emb-type', bold_max(ntlm_df).rename(columns=metrics).drop('Unjudged@20',axis='columns').to_latex(escape=False))

In [ ]:
ntlm_fig = plot_tune_1d_comp(['base'] + abbrev, RERANK_METRICS, [[base_df for x in range(20)]] + [[x.mean() for x in y] for y in ntlm_runs], 1, 20, 1, legend_x=0.965, legend_y=0.2, styles=['--'], ylims=RERANK_YLIMS)
# ntlm_fig.savefig('figures/ausnl-ntlm.pdf')

In [ ]:
best_qry = load_dfs(config.AUS_QREL_PATH, config.AUS_REL_LEVEL, '', [grid_name.format(x, 'RBP') for x in ['cbow-centroid', 'skip-max']], per_query=True)

In [ ]:
(best_qry[1]-base_qry).sort_values('rbp@0.80')[:5]